In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/heart-dissease/heart_Disease.csv')

In [ ]:
df.head()

In [ ]:
df.corr()

We some correlation, but all relatives with target feature is small.

In [ ]:
df.dtypes

In [ ]:
df.target.value_counts()

Not enought samples. Let's make test set with 100 samples and generate new samples from the remaining ones.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis = 1), df.target, test_size = 100)

In [ ]:
from sklearn.utils import resample

In [ ]:
X_res, y_res = resample(df.drop('target', axis = 1), df.target, n_samples = 1000, random_state = 42)

Have set with 1000 samples, but they are copies each other.

Let's use SMOTE for generate similiar samples.

In [ ]:
from collections import Counter

In [ ]:
Counter(y_res)

In [ ]:
X_positive = X_res[y_res == 1]

In [ ]:
X_negative = X_res[y_res == 0]

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote_pos = SMOTE(random_state = 42)

In [ ]:
X_positive.loc[:,'target'] = 1
X_negative.loc[:,'target'] = 0

In [ ]:
Counter(X_positive.target)

In [ ]:
X_for_pos = X_positive.append(X_train[y_train == 0])

In [ ]:
X_for_pos.target = X_for_pos.target.fillna(0)

In [ ]:
X_for_neg = X_negative.append(X_train[y_train == 1])

In [ ]:
X_for_neg.target = X_for_neg.target.fillna(1)

In [ ]:
Counter(X_for_pos.target)

In [ ]:
Counter(X_for_neg.target)

In [ ]:
X_pos_res, y_pos_res = smote_pos.fit_resample(X_for_pos.drop('target', axis = 1), X_for_pos.target)

In [ ]:
smote_neg = SMOTE(random_state = 42)

In [ ]:
X_neg_res, y_neg_res = smote_neg.fit_resample(X_for_neg.drop('target', axis = 1), X_for_neg.target)

In [ ]:
Counter(y_neg_res)

In [ ]:
Counter(y_pos_res)

In [ ]:
X_pos_res['target'] = y_pos_res
X_neg_res['target'] = y_neg_res

In [ ]:
df_train = X_pos_res.append(X_neg_res)

In [ ]:
Counter(df_train.target)

Okay, now we have train set with 2000 samples.

In [ ]:
X_train = df_train.drop('target', axis = 1)
y_train = df_train.target

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Let's try decision tree.

In [ ]:
tree = DecisionTreeClassifier(random_state = 42)
tree.fit(X_train, y_train)

In [ ]:
tree.score(X_train, y_train)

It's high accuracy. Maybe we have overfitting.

In [ ]:
tree.score(X_test, y_test)

bias = 0, variance = 0.03. We have good working model. Let's try make it a little better with gridsearch

In [ ]:
X_train.columns

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {'max_depth' : [None,4, 10, 15, 20, 25,30,35,40,45,50],\
         'min_samples_leaf' : [1, 3, 4, 6, 8, 10, 12,15,18, 20, 25],\
         'max_leaf_nodes' : [None, 2, 4, 6, 8, 10, 12, 15],\
          'max_features' : [None, 'auto', 'log2', 'sqrt'],\
         }

In [ ]:
clf = GridSearchCV(tree, params)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

In [ ]:
best_tree = clf.best_estimator_

In [ ]:
best_tree.score(X_train, y_train)

In [ ]:
best_tree.score(X_test, y_test)

Okay, best tree have better accuracy 1% better.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

Should check another metrics, maybe it isn't so good as look.

In [ ]:
y_pred = best_tree.predict(X_test)

In [ ]:
print('roc_auc: ', roc_auc_score(y_test, y_pred))
print('precision: ', precision_score(y_test, y_pred))
print('recall: ', recall_score(y_test, y_pred))
print('f1: ', f1_score(y_test, y_pred))

All metrics are good.